# 1. SageMaker Training For AutoGluon

### 작업 실행 시 필요 라이브러리 import

In [ ]:
!pip install -U sagemaker-experiments

In [2]:
!pip install sagemaker ipywidgets --upgrade --quiet

In [ ]:
!pip install autogluon

In [6]:
import mxnet
mxnet.__version__

'1.7.0'

In [7]:
import pandas as pd

In [83]:
import sys
import os

In [8]:
from sklearn.model_selection import train_test_split #splitting the dataset
from autogluon.tabular import TabularDataset, TabularPredictor #to handle tabular data and train models

In [4]:
import sagemaker, boto3, json
from sagemaker import get_execution_role

aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

### Dataset

본 예제는 [ETDataset](https://github.com/zhouhaoyi/ETDataset)을 사용합니다. 데이터 셋은 하위와 같이 Github 리포지토리를 클론해 다운로드 받을 수 있도록 합니다.

In [5]:
!git clone https://github.com/zhouhaoyi/ETDataset.git

fatal: destination path 'ETDataset' already exists and is not an empty directory.


In [9]:
df = pd.read_csv('./ETDataset/ETT-small/ETTh1.csv')

In [10]:
df = df.drop(['date'], axis=1)

In [11]:
df_train,df_test=train_test_split(df,test_size=0.33,random_state=1)
df_train.shape,df_test.shape

((11671, 7), (5749, 7))

In [12]:
df_train.head()

,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
15162,6.765,1.541,3.553,0.320,3.198,0.975,12.662000
16013,15.004,2.545,10.518,0.604,4.630,1.157,11.607000
2764,10.918,3.215,7.285,2.772,3.594,-0.487,17.445999
1755,8.975,4.756,7.356,3.411,1.340,0.701,19.979000
695,19.357,2.813,13.752,-0.071,5.970,1.492,40.450001


In [13]:
test_data=df_test.drop(['OT'],axis=1)
test_data.head()

,HUFL,HULL,MUFL,MULL,LUFL,LULL
14198,-16.209,2.880,-18.905001,1.244,2.467,0.670
13512,12.056,2.880,8.742000,1.635,2.802,0.914
2069,12.592,3.550,10.625000,2.310,1.706,0.701
8308,8.439,1.474,5.757000,1.741,1.919,-0.609
16448,11.922,4.890,7.178000,2.345,5.056,1.614


### AutoGluon 

In [ ]:
predictor= TabularPredictor(label ='OT').fit(train_data = df_train, verbosity = 2,presets='best_quality', hyperparameters = {
               # 'NN': {'num_epochs': 500},
               'GBM': {},
               'CAT': {},
               'RF': {},
               'XT': {},
               'KNN': {},
#                'custom': ['GBM'],
})

In [ ]:
predictor.fit_summary()

In [19]:
predictor.leaderboard(df_train, silent=True)


,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,ExtraTrees_BAG_L1,-2.373084,-6.390254,0.425219,0.474958,0.912591,0.425219,0.474958,0.912591,1,True,5
1,RandomForest_BAG_L1,-2.377621,-6.407553,0.243681,0.469496,1.414026,0.243681,0.469496,1.414026,1,True,3
2,WeightedEnsemble_L2,-3.472599,-6.328754,1.273522,1.135001,31.640994,0.004006,0.000597,0.240671,2,True,6
3,CatBoost_BAG_L2,-3.812389,-6.338275,1.301823,1.159677,36.433794,0.032307,0.025274,5.033472,2,True,9
4,WeightedEnsemble_L3,-3.863211,-6.331895,1.551975,1.650985,37.656726,0.001958,0.000614,0.206405,3,True,11
5,LightGBM_BAG_L2,-3.937479,-6.401470,1.357438,1.160436,34.295342,0.087922,0.026032,2.895019,2,True,7
6,ExtraTrees_BAG_L2,-4.205004,-6.405582,1.517710,1.625097,32.416850,0.248194,0.490694,1.016527,2,True,10
7,RandomForest_BAG_L2,-4.316848,-6.470987,1.515311,1.632674,34.621463,0.245796,0.498271,3.221140,2,True,8
8,LightGBM_BAG_L1,-5.231914,-6.470730,0.440583,0.056886,4.771393,0.440583,0.056886,4.771393,1,True,2
9,CatBoost_BAG_L1,-5.433174,-6.454864,0.055013,0.028192,24.290092,0.055013,0.028192,24.290092,1,True,4


In [20]:
predictor.feature_importance(data=df_train)

Computing feature importance via permutation shuffling for 6 features using 1000 rows with 3 shuffle sets...
	15.98s	= Expected runtime (5.33s per shuffle set)
	5.08s	= Actual runtime (Completed 3 of 3 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
LULL,4.186301,0.239801,0.000546,3,5.560385,2.812217
LUFL,3.194513,0.103489,0.000175,3,3.787515,2.601511
MUFL,2.999799,0.057395,0.000061,3,3.328682,2.670917
MULL,2.917758,0.132664,0.000344,3,3.677939,2.157576
HULL,2.334510,0.060790,0.000113,3,2.682843,1.986177
HUFL,1.878686,0.106805,0.000538,3,2.490693,1.266679


In [22]:
y_pred = predictor.predict(test_data)
y_pred=pd.DataFrame(y_pred,columns=['OT'])
y_pred #print the DataFrame

,OT
14198,7.291081
13512,8.206678
2069,16.144300
8308,15.873062
16448,11.764364
...,...
476,15.841290
5004,9.059195
10954,18.083954
16212,10.732670


In [23]:
predictor.evaluate(df_test)

Evaluation: root_mean_squared_error on test data: -6.391080107445289
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -6.391080107445289,
    "mean_squared_error": -40.84590493978301,
    "mean_absolute_error": -4.507494016361431,
    "r2": 0.456628203348066,
    "pearsonr": 0.6770374919797111,
    "median_absolute_error": -3.2320613861083984
}


{'root_mean_squared_error': -6.391080107445289,
 'mean_squared_error': -40.84590493978301,
 'mean_absolute_error': -4.507494016361431,
 'r2': 0.456628203348066,
 'pearsonr': 0.6770374919797111,
 'median_absolute_error': -3.2320613861083984}

### SageMaker

In [41]:
from sagemaker import image_uris, model_uris, script_uris

train_model_id, train_model_version, train_scope = "autogluon-regression-ensemble", "*", "training"

training_instance_type = "ml.m5.12xlarge"

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=train_model_id,
    model_version=train_model_version,
    image_scope=train_scope,
    instance_type=training_instance_type,
)
# Retrieve the training script
train_source_uri = script_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, script_scope=train_scope
)
# Retrieve the pre-trained model tarball to further fine-tune. In tabular case, however, the pre-trained model tarball is dummy and fine-tune means training from scratch.
train_model_uri = model_uris.retrieve(
    model_id=train_model_id, model_version=train_model_version, model_scope=train_scope
)

image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


In [56]:
!aws s3api get-object --bucket jumpstart-cache-prod-ap-northeast-2 --key source-directory-tarballs/autogluon/transfer_learning/regression/v1.0.1/sourcedir.tar.gz sourcedir.tar.gz

{
    "AcceptRanges": "bytes",
    "LastModified": "Sat, 30 Jul 2022 18:40:11 GMT",
    "ContentLength": 3198,
    "ETag": "\"6d151a8ce01643d903391df6f5058390\"",
    "VersionId": "sy1FGcdI6WJN9p7oU_vrS3yeHB30WPI0",
    "ContentType": "binary/octet-stream",
    "Metadata": {}
}


In [59]:
!mkdir source 

mkdir: cannot create directory ‘source’: File exists


In [61]:
!cd source && tar -zxvf sourcedir.tar.gz

__init__.py
constants/
constants/__init__.py
constants/__pycache__/
constants/__pycache__/__init__.cpython-37.pyc
constants/__pycache__/constants.cpython-37.pyc
constants/constants.py
transfer_learning.py
version
__pycache__/
__pycache__/__init__.cpython-37.pyc


In [67]:
# Sample training data is available in this bucket
training_data_bucket = f"yudong-data"
training_data_prefix = "training-datasets/tabular_regression/train"

# training_dataset_s3_path = f"s3://{training_data_bucket}/{training_data_prefix}"

output_prefix = "training-datasets/tabular_regression"

s3_output_location = f"s3://{training_data_bucket}/{output_prefix}/output"

In [68]:
df_train.to_csv('train.csv', index=False)

In [69]:
training_dataset_s3_path = session.upload_data('train.csv', key_prefix=training_data_prefix, bucket=training_data_bucket)

In [71]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for training the model
hyperparameters = hyperparameters.retrieve_default(
    model_id=train_model_id, model_version=train_model_version
)

# [Optional] Override default hyperparameters with custom values
hyperparameters["auto_stack"] = "True"
print(hyperparameters)

{'eval_metric': 'auto', 'presets': 'medium_quality', 'auto_stack': 'True', 'num_bag_folds': '0', 'num_bag_sets': '1', 'num_stack_levels': '0', 'refit_full': 'False', 'set_best_to_refit_full': 'False', 'save_space': 'False', 'verbosity': '2'}


In [80]:
training_instance_type = 'local'
max_run = 1*60*60

use_spot_instances = False
if use_spot_instances:
    max_wait = 1*60*60
else:
    max_wait = None

In [84]:
if training_instance_type == 'local':
    from sagemaker.local import LocalSession
    
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    local_data_path = "file://" + os.getcwd().replace('/lab_1_training', '') + "/"
    
    print(local_data_path)
    
    data_channels = {
        "inputdata": local_data_path, 
    }
    
else:
    sess = boto3.Session()
    sagemaker_session = sagemaker.Session()
    sm = sess.client('sagemaker')
    
    data_channels = {
        "inputdata": data_path, 
    }

file:///home/ec2-user/SageMaker/mlops-ramp/mlops-for-autogluon-step-fn/1_sagemaker_training/


In [101]:
hyperparameters

{'eval_metric': 'auto',
 'presets': 'medium_quality',
 'auto_stack': 'True',
 'num_bag_folds': '0',
 'num_bag_sets': '1',
 'num_stack_levels': '0',
 'refit_full': 'False',
 'set_best_to_refit_full': 'False',
 'save_space': 'False',
 'verbosity': '2',
 'sagemaker_submit_directory': 'file:///home/ec2-user/SageMaker/mlops-ramp/mlops-for-autogluon-step-fn/1_sagemaker_training/source',
 'sagemaker_program': 'transfer_learning.py',
 'sagemaker_container_log_level': 20,
 'sagemaker_job_name': 'autogluon-autogluon-regression-ensemble-2022-08-15-14-17-33-339',
 'sagemaker_region': 'ap-northeast-2'}

In [109]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base

training_job_name = name_from_base(f"autogluon-{train_model_id}-training")

# Create SageMaker Estimator instance
tabular_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir='source',
#     model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
)

# Launch a SageMaker Training job by passing s3 path of the training data
tabular_estimator.fit({"train": training_dataset_s3_path}, logs=False, job_name=training_job_name, wait=False)

Defaulting to the only supported framework/algorithm version: latest.
Ignoring unnecessary instance type: None.
Creating training-job with name: autogluon-autogluon-regression-ensemble-2022-08-15-20-29-33-019
Starting training job
No AWS credentials found in session but credentials from EC2 Metadata Service are available.
docker compose file: 
networks:
  sagemaker-local:
    name: sagemaker-local
services:
  algo-1-kyfef:
    command: train
    container_name: s25n1fxoqk-algo-1-kyfef
    environment:
    - '[Masked]'
    - '[Masked]'
    image: 763104351884.dkr.ecr.ap-northeast-2.amazonaws.com/autogluon-training:0.4.0-cpu-py38
    networks:
      sagemaker-local:
        aliases:
        - algo-1-kyfef
    stdin_open: true
    tty: true
    volumes:
    - /tmp/tmp0za9twmc/algo-1-kyfef/input:/opt/ml/input
    - /tmp/tmp0za9twmc/algo-1-kyfef/output/data:/opt/ml/output/data
    - /tmp/tmp0za9twmc/algo-1-kyfef/output:/opt/ml/output
    - /tmp/tmp0za9twmc/model:/opt/ml/model
    - /opt/ml/

Creating s25n1fxoqk-algo-1-kyfef ... 
Creating s25n1fxoqk-algo-1-kyfef ... done
Attaching to s25n1fxoqk-algo-1-kyfef
s25n1fxoqk-algo-1-kyfef | 2022-08-15 20:29:34,753 sagemaker-training-toolkit INFO     Imported framework sagemaker_mxnet_container.training
s25n1fxoqk-algo-1-kyfef | 2022-08-15 20:29:34,755 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
s25n1fxoqk-algo-1-kyfef | 2022-08-15 20:29:34,755 sagemaker-training-toolkit INFO     Failed to parse hyperparameter eval_metric value auto to Json.
s25n1fxoqk-algo-1-kyfef | Returning the value itself
s25n1fxoqk-algo-1-kyfef | 2022-08-15 20:29:34,755 sagemaker-training-toolkit INFO     Failed to parse hyperparameter presets value medium_quality to Json.
s25n1fxoqk-algo-1-kyfef | Returning the value itself
s25n1fxoqk-algo-1-kyfef | 2022-08-15 20:29:34,755 sagemaker-training-toolkit INFO     Failed to parse hyperparameter auto_stack value True to Json.
s25n1fxoqk-algo-1-kyfef | Returning the value itself

###  실험 결과 보기
위의 실험한 결과를 확인 합니다.
- 각각의 훈련잡의 시도에 대한 훈련 사용 데이터, 모델 입력 하이퍼 파라미터, 모델 평가 지표, 모델 아티펙트 결과 위치 등의 확인이 가능합니다.
- **아래의 모든 내용은 SageMaker Studio 를 통해서 직관적으로 확인이 가능합니다.**

In [ ]:
!rm -rf ./autogluon/
!mkdir -p ./autogluon/result
!aws s3 cp {ag_estimator.model_data} ./autogluon/

In [ ]:
!ls -alF ./autogluon/model.tar.gz

In [ ]:
!tar -xzf ./autogluon/model.tar.gz -C ./autogluon/result/

### Endpoint Deployment

In [ ]:
instance_type = "ml.m5.2xlarge"
# instance_type = 'local'

In [ ]:
if instance_type == 'local':
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
else:
    sess = boto3.Session()
    sagemaker_session = sagemaker.Session()

In [ ]:
model = AutoGluonInferenceModel(
    source_dir=os.getcwd() + "/src",
    entry_point="autogluon_serve.py",
    model_data=ag_estimator.model_data,
    instance_type=instance_type,
    role=role,
    sagemaker_session=sagemaker_session,
    # region=region,
    framework_version="0.4",
    py_version="py38",
    predictor_cls=AutoGluonTabularPredictor
)

In [ ]:
from sagemaker.serializers import CSVSerializer

predictor = model.deploy(
    initial_instance_count=1, serializer=CSVSerializer(), instance_type=instance_type
)

### Predict on unlabeled test data

Remove target variable (`fraud`) from the data and get predictions for a sample of 100 rows using the deployed endpoint.

In [ ]:
df = pd.read_csv("../data/dataset/test.csv")
data = df.drop(columns="fraud")[:100].values

In [ ]:
preds = predictor.predict(data)
pred_df = pd.DataFrame(json.loads(preds))

In [ ]:
pred_df['fraud'].reset_index(drop=True, inplace=True)
df["fraud"][:len(pred_df)].reset_index(drop=True, inplace=True)

In [ ]:
p = pd.DataFrame({"preds": pred_df['fraud'], "actual": df["fraud"][: len(pred_df)]})
p.head()

In [ ]:
print(f"{(p.preds==p.actual).astype(int).sum()}/{len(p)} are correct")

### Cleanup Endpoint

In [ ]:
# predictor.delete_endpoint()

# Batch Transform

학습된 모델을 호스트된 엔드포인트에 배포하는 것은 출시 이후 SageMaker에서 사용할 수 있으며 웹 사이트나 모바일 앱과 같은 서비스에 실시간 예측을 제공하는 좋은 방법입니다. 그러나 지연 시간을 최소화하는 것이 문제가 되지 않는 대규모 데이터 세트에서 학습된 모델에서 예측을 생성하는 것이 목표라면 배치 변환 기능이 더 쉽고, 더 확장 가능하며, 더 적절할 수 있다.

[Read more about Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html).

In [ ]:
instance_type = "ml.m5.2xlarge"

In [ ]:
# model = AutoGluonInferenceModel(
#     source_dir=os.getcwd() + "/src",
#     entry_point="autogluon_serve.py",
#     model_data=ag_estimator.model_data,
#     instance_type=instance_type,
#     role=role,
#     sagemaker_session=sagemaker_session,
#     region=region,
#     framework_version="0.4",
#     py_version="py38",    
#     predictor_cls=AutoGluonTabularPredictor,
# )

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=instance_type,
    strategy="MultiRecord",
    max_payload=6,
    max_concurrent_transforms=1,
    output_path=output_path,
    accept="application/json",
    assemble_with="Line",
)


Prepare data for batch transform

In [ ]:
pd.read_csv(f"../data/dataset/test.csv")[:100].to_csv("../data/dataset/test_no_header.csv", header=False, index=False)

In [ ]:
test_input = transformer.sagemaker_session.upload_data(
    path=os.path.join("../data/dataset", "test_no_header.csv"), key_prefix=f"{bucket}/autogluon/dataset"
)
test_input

In [ ]:
transformer.transform(
    test_input,
    input_filter="$[1:]",  # filter-out target variable
    split_type="Line",
    content_type="text/csv",
    output_filter="$['fraud']",  # keep only prediction class in the output
)

transformer.wait()

batch transform 결과를 다운로드 받습니다.

In [ ]:
!rm -rf ./autogluon_batch_result
!mkdir ./autogluon_batch_result

In [ ]:
transformer.output_path

In [ ]:
!aws s3 cp {transformer.output_path}/test_no_header.csv.out ./autogluon_batch_result/

In [ ]:
p = pd.concat(
    [
        pd.read_json("./autogluon_batch_result/test_no_header.csv.out", orient="index")
        .sort_index()
        .rename(columns={0: "preds"}),
        pd.read_csv("../data/dataset/test.csv")[["fraud"]].iloc[:100].rename(columns={"fraud": "actual"}),
    ],
    axis=1,
)
p.head()

In [ ]:
print(f"{(p.preds==p.actual).astype(int).sum()}/{len(p)} are correct")

###  Processing Evaluation 하기
SageMaker Processing을 이용하여 Evalution을 수행하는 코드를 동작할 수 있습니다. MLOps에서 Processing을 적용하면 전처리, Evaluation 등을 serverless로 동작할 수 있습니다.

In [ ]:
from sagemaker.processing import FrameworkProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.estimator import Framework

In [ ]:
instance_count = 1
instance_type = "ml.m5.large"
# instance_type = 'local'

In [ ]:
from sagemaker import image_uris

image_uri = image_uris.retrieve(
    "autogluon",
    region=region,
    version="0.4",
    py_version="py38",
    image_scope="training",
    instance_type=instance_type,
)
image_uri


In [ ]:
script_eval = FrameworkProcessor(
    AutoGluonFramework,
    framework_version="0.4",
    role=role,
    py_version="py38",
    image_uri=image_uri,
    instance_type=instance_type,
    instance_count=instance_count
)

In [ ]:
detect_outputpath = f's3://{bucket}/autogluon/processing'

In [ ]:
source_dir='src'

if instance_type == 'local':
    from sagemaker.local import LocalSession
    from pathlib import Path
    
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    source_dir = f'{Path.cwd()}/src'
    s3_test_path=f'../data/dataset/test.csv'
else:
    sagemaker_session = sagemaker.Session()
    s3_test_path = data_path + '/test.csv'

In [ ]:
create_experiment(experiment_name)
job_name = create_trial(experiment_name)

script_eval.run(
    code="autogluon_evaluation.py",
    source_dir=source_dir,
    inputs=[ProcessingInput(source=s3_test_path, input_name="test_data", destination="/opt/ml/processing/test"),
            ProcessingInput(source=ag_estimator.model_data, input_name="model_weight", destination="/opt/ml/processing/model")
    ],
    outputs=[
        ProcessingOutput(source="/opt/ml/processing/output", output_name='evaluation', destination=detect_outputpath + "/" + job_name),
    ],
    job_name=job_name,
    experiment_config={
        'TrialName': job_name,
        'TrialComponentDisplayName': job_name,
    },
    wait=False
)

In [ ]:
script_eval.latest_job.wait()

### Code repository 생성 및 push
현재 사용하는 노트북의 iam role에 IAMFullAccess을 추가한 이후에 아래 작업을 수행합니다.

In [ ]:
from sagemaker import get_execution_role

In [ ]:
iam_client = boto3.client('iam')

role=get_execution_role()
base_role_name=role.split('/')[-1]

iam_client.attach_role_policy(
    RoleName=base_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AWSCodeCommitFullAccess'
)

In [ ]:
codecommit = boto3.client('codecommit')
repository_name = 'autogluon_code'

try:
    response = codecommit.create_repository(
        repositoryName=repository_name,
        repositoryDescription='Data Scientists share their training code using this Repository'
    )
except:
    
    print("Repository already exists")
    response = codecommit.get_repository(
        repositoryName=repository_name
    )

In [ ]:
codecommit_repo = response['repositoryMetadata']['cloneUrlHttp']
codecommit_repo

In [ ]:
!git init
!git remote add repo_codecommit $codecommit_repo
!git checkout -b main
!git add ./config ./src ./1.SageMaker-Training+Experiments+Processing-AutoGluon.ipynb
!git commit -m "autogluon-update"
!git push --set-upstream repo_codecommit main